In [1]:
### Libraries
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
### Loading Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.load(r'left_x_train.npy')
x_test = np.load(r'left_x_test.npy')

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('Training data shape: ', x_train.shape)
print('Testing data shape : ', x_test.shape)

Training data shape:  (60000, 28, 28)
Testing data shape :  (10000, 28, 28)


In [3]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
### Training AE
from keras.callbacks import TensorBoard
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 19s 317us/step - loss: 0.1062 - val_loss: 0.0707
Epoch 2/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0662 - val_loss: 0.0616
Epoch 3/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0591 - val_loss: 0.0553
Epoch 4/50
60000/60000 [==============================] - 17s 284us/step - loss: 0.0549 - val_loss: 0.0520
Epoch 5/50
60000/60000 [==============================] - 17s 284us/step - loss: 0.0520 - val_loss: 0.0515
Epoch 6/50
60000/60000 [==============================] - 17s 284us/step - loss: 0.0503 - val_loss: 0.0488
Epoch 7/50
60000/60000 [==============================] - 17s 284us/step - loss: 0.0490 - val_loss: 0.0481
Epoch 8/50
60000/60000 [==============================] - 17s 288us/step - loss: 0.0478 - val_loss: 0.0482
Epoch 9/50
60000/60000 [======================

In [5]:
### Get Encoded features fron AE
encoder = Model(input_img, encoded)
encoded_imgs_test = encoder.predict(x_test)
encoded_imgs_train = encoder.predict(x_train)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 4, 4, 8)
(60000, 4, 4, 8)


In [6]:
encoded_imgs_test[0]

array([[[0.        , 1.8651708 , 0.43314257, 1.3234646 , 0.9033291 ,
         0.3255819 , 1.8197528 , 0.69088566],
        [0.5463997 , 2.0978804 , 0.7307594 , 0.71464443, 0.48400748,
         1.4261339 , 1.6030585 , 0.6522656 ],
        [1.1289582 , 0.10561787, 0.23857817, 0.41181818, 0.18166311,
         1.0215348 , 1.4930311 , 0.7148983 ],
        [0.26402307, 0.        , 0.04888347, 0.32963136, 0.7024624 ,
         0.30097514, 0.6359995 , 0.87245405]],

       [[0.7589229 , 1.2272817 , 1.3372182 , 1.6736633 , 1.3918722 ,
         0.13132855, 0.99513733, 1.0659105 ],
        [1.3451583 , 1.7389289 , 1.462357  , 2.1788864 , 0.96532047,
         1.9761679 , 1.7042996 , 1.0706024 ],
        [1.7947378 , 0.25978497, 0.728865  , 1.0082989 , 0.        ,
         2.2298496 , 1.2796891 , 0.33708343],
        [0.26395306, 0.        , 0.0488143 , 0.32971856, 0.70250314,
         0.3009434 , 0.6361264 , 0.87250465]],

       [[0.27118266, 0.12915725, 0.48774526, 1.0489821 , 0.4238119 ,
       

In [7]:
encoded_imgs_test = encoded_imgs_test.reshape(10000,128)
encoded_imgs_train = encoded_imgs_train.reshape(60000,128)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 128)
(60000, 128)


In [8]:
np.save('left_test.npy',encoded_imgs_test)
np.save('left_train.npy',encoded_imgs_train)